# Welcome to Data Analysis!
Here you will pull out the main details of the particles you have tracked. 
There are three main sections:
* Diffusion 
* Conformation
* Ergodicity
***
To jump to the Diffusion functions click [here](#Diffusion-Methods).

To jump to the Conformational functions click [here](#Conformational-Methods)

To jump to Ergodicity functions click [here](#Ergodicity)

### Import Packages

In [2]:
import cv2
print cv2.__version__

3.4.3


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns               #version 0.9.0
import warnings
import pims                         #version 0.4.1
import trackpy as tp                #version 0.4.1
import pandas as pd                 #version 0.23.4
import os
import csv                          #version 1.0
import numpy as np                  #version 1.15.1
import imageio                      #version 2.4.1
import pickle                       #revision: 72223
import time
import datetime
import math
from collections import defaultdict
import cv2                          #version 3.4.3
import sys

sns.set_style("darkgrid")
%matplotlib inline

warnings.filterwarnings("ignore")

In [4]:
print pickle.__version__

$Revision: 72223 $


## Loading Files

In [3]:
def loadIndividualDataset(f_root, subdir_name):
    ''' Loads an individual dataset
    
    Input:
        f_root = root directory for data
        subdir_name = name of the subdirectory containing data
        frames (optional) = option to grab frames data
        
    Output:
        centers = dictionary containing data for the individual dataset
    '''
    sub_dir = os.path.join(f_root, subdir_name)
    
    centers_loc = os.path.join(sub_dir, 'centers.p')
    centers = pickle.load(open(centers_loc, "rb" ))

    return centers

def loadMultipleDatasets(f_root, subdir_names):  
    ''' Loads multiple datasets into dict
    
    Input:
        f_root = root directory for data
        subdir_names = list of subdirectories to pull data for
        frames (optional) = option to grab frames data
    '''
    return {subdir_name: loadIndividualDataset(f_root, subdir_name) \
            for subdir_name in subdir_names}


## Diffusion Methods
* Get differences between frames
* Plot histogram of differences



In [4]:
def getDiffs(datasets):
    ''' Takes dictionary (output of `loadMultipleDatasets`) and iterates through each particle.
    For each particle, sweeps through and finds all diffs between frames (for each possible window
    size) and adds to master dict (one for each axis)
    
    Input:
        datasets = dictionary of datasets, output of `loadMultipleDatasets`
        
    Output:
        dictionary of diffs for each window size (one for each axis)
    '''
    diffs_x = defaultdict(list)
    diffs_y = defaultdict(list)
    diffs_comb = defaultdict(list)
    
    # iterate through datasets
    for dataset, data in datasets.iteritems():
        # iterate through videos
        for video, particles in data['all_centers_dict_normed'].items():
            # iterate through particles
            for particle, center_data in particles.items():
                x_centers = center_data['x']
                y_centers = center_data['y']
                
                # iterate through possible sliding window sizes
                for window in range(1, len(x_centers)):
                    for ind_start in range(len(x_centers)-window):
                        diffs_x[window].append(x_centers[window+ind_start] - x_centers[ind_start]) 
                        diffs_y[window].append(y_centers[window+ind_start] - y_centers[ind_start])
                        diffs_comb[window].append(diffs_x[window][-1])
                        diffs_comb[window].append(diffs_y[window][-1])

    return {'x': diffs_x,
           'y': diffs_y,
           'comb': diffs_comb}

""" Command to plot """
def plotIndividualHistOfDiffs(diff_dict, window, binsize, axis):
    ''' Plots histogram of diffs for particular window size, axis
    
    Input:
        diff_dict = axis dict of differences
        window = size of the window to plot hist (number of frames)
        axis = string of axis (for labeling)
        
    Output:
        None, plots individual histogram
    '''
    plt.figure()
    plt.hist(diff_dict[window], bins = binsize);
    plt.title('%s-diffs: %d frames' % (axis, window), fontsize=15)
    plt.xlabel('pixels', fontsize=15)
    plt.ylabel('count', fontsize=15)
    plt.legend(fontsize=15)
    
    plt.figure()
    plt.hist(diff_dict[window], bins = binsize);
    plt.title('%s-diffs: %d frames' % (axis, window), fontsize=15)
    plt.xlabel('pixels', fontsize=15)
    plt.ylabel('count', fontsize=15)
    plt.legend(fontsize=15)
    plt.yscale('log')
    
    
def plotAllHistOfDiffs(diffs, window,binsize):
    ''' Calls `plotIndividualHistOfDiffs` for each axis (x, y, combined) for a 
    particular window
    
    Input:
        diffs = dictionary of diffs (one for each axis)
        window = size of the window to plot hists (number of frames)
        
    Output:
        None, plots individual histograms
    '''
    for axis, axis_dict in diffs.items():
        plotIndividualHistOfDiffs(axis_dict, window, binsize, axis)
def mean(vals_list):
    ''' Returns mean of a list of values. Because this doesn't exist
    natively in python!
    
    Input:
        vals_list = list of values
        
    Output:
        single value (float), mean of the list
    '''
    return sum(vals_list)/float(len(vals_list))

def calculateMSDs(diffs_dict):
    ''' Calculates the msd values for each time-frame, for each axis
    
    Input:
        diffs_dict = dictionary of diffs (one for each axis), output of `getDiffs`
        
    Output:
        diffs_squared_dict = dictionary of MSD values {frame: pixels^2} for each axis
    '''
    
    diffs_squared_dict = {'x': {0: 0}, 'y': {0: 0}, 'comb': {0: 0}}
    for axis, axis_dict in diffs_dict.iteritems():
        for window, diffs in axis_dict.iteritems():
            diffs_squared_dict[axis][window] = mean([diff**2 for diff in diffs])
        
    return diffs_squared_dict

def plotIndividualMsd(msd_dict, axis, line_type='-o'):
    ''' Plots MSD for individual axis
    
    Input:
        msd_dict = MSD values all axes (output of `calculateMSDs`)
        axis = string of the axis to plot
        line_type (optional) = line type for the plot
        
    Output:
        None, plots the individual MSD
    '''
    
    plt.figure();
    
    vals = sorted([(k, v) for k, v in msd_dict.items()])
    x = [v[0] for v in vals]
    y = [v[1] for v in vals]
    
    plt.plot(x, y, line_type)
    plt.title('MSD for axis: %s' % axis, fontsize=15)
    plt.xlabel('frames', fontsize=15)
    plt.ylabel('pixels^2', fontsize=15)
    
def plotAllMsds(msd_dict, line_type='-o'):
    for axis, axis_dict in msd_dict.items():
        plotIndividualMsd(axis_dict, axis, line_type=line_type)
        
def linearFitIndividualMsd(axis_dict, axis, start_frame=0, end_frame=-1, plot=True, line_type='-o'):
    if end_frame == -1:
        end_frame = len(axis_dict.values()) - 1
        
    # ensure proper sorting of x, y
    vals = sorted([(k, v) for k, v in axis_dict.items()])
    x = [v[0] for v in vals][start_frame:end_frame]
    y = [v[1] for v in vals][start_frame:end_frame]
    
    linear_fits = np.polyfit(x,y,1)
    linear_fits_fn = np.poly1d(linear_fits) 

    if plot:
        plotIndividualMsd(axis_dict, axis, line_type=line_type)
        plt.plot(x, linear_fits_fn(x), '-');
        
    return linear_fits

def linearFitAllAxes(msd_dict, start_frame=0, end_frame=-1, plot=True, line_type='-o', linewidth=2):
    linear_fits = {}
    for axis, axis_dict in msd_dict.items():
        linear_fits[axis] = linearFitIndividualMsd(\
            axis_dict, axis, start_frame=start_frame, end_frame=end_frame, plot=plot, line_type=line_type)
        
    return linear_fits

def logLogMsdData(msd_dict):
    msd_dict_ll = {}
    for axis, axis_dict in msd_dict.items():
        msd_dict_ll[axis] = {}
        for window, msd_val in axis_dict.items():
            if window == 0:
                continue
            
            msd_dict_ll[axis][math.log(window)] = math.log(msd_val)
    
    return msd_dict_ll

def calculateIndividualMsdStats(axis_dict, axis_dict_ll, start_frame=0, end_frame=-1):
    stats = {'start_frame': start_frame, 'end_frame': end_frame}
    stats['linear_fit'] = linearFitIndividualMsd(\
            axis_dict, '', start_frame=start_frame, end_frame=end_frame, plot=False)
    
    stats['linear_fit_ll'] = linearFitIndividualMsd(\
            axis_dict_ll, '', start_frame=start_frame, end_frame=end_frame, plot=False)
    
    return stats

def calculateAllMsdStats(msd_dict, start_frame=0, end_frame=-1):
    msd_dict_ll = logLogMsdData(msd_dict)

    all_stats = {}
    for axis, axis_dict in msd_dict.iteritems():
        all_stats[axis] = calculateIndividualMsdStats(\
            axis_dict, msd_dict_ll[axis], start_frame=start_frame, end_frame=end_frame)
        
    return all_stats

def loadDatasetsAndCalculateMsdStats(f_root, subdir_names, start_frame=0, end_frame=-1):
    datasets = loadMultipleDatasets(f_root, subdir_names)
    diffs_dict = getDiffs(datasets)

    msd_dict = calculateMSDs(diffs_dict)
    msd_dict_ll = logLogMsdData(msd_dict)

    all_stats = calculateAllMsdStats(msd_dict, start_frame=start_frame, end_frame=end_frame)
    return all_stats

## Diffusion Analysis: Load and run

In [37]:

Conformation = "Actin"
Condition = "Ring"
date="20181205"
mass_used1='2500'
mass_used2='3000'


#Cloud_Location = "\\\ANDERSONLAB\\AndersonLab"

'''Enter the names of your files below, normally in day_Condition_Minmass format'''
#group="10"


subdir_names=(["JG_test_1_"
              ])

In [38]:
#f_root = Cloud_Location+"\\"+User+"\\DATA\\DNA_in_XL_systems\\Lightsheet\\"+Conformation+"\\"+Condition+"\\Data\\"
f_root="F:\\19_12_3_XLMicrotubules\\"

In [39]:
print f_root
print subdir_names

F:\19_12_3_XLMicrotubules\
['JG_test_1_']


## Diffusion Analysis: Go through sequentially
### **Start with calculating all MSD data below**

In [47]:
datasets = loadMultipleDatasets(f_root, subdir_names)
diffs_dict = getDiffs(datasets)
#binsize=45

msd_dict = calculateMSDs(diffs_dict)
#msd_dict_ll = logLogMsdData(msd_dict)

To jump to the MSD plotting (linear-linear) click [here](#2.-Plotting-MSDs:-linear-and-log-log)

KeyError: 0

In [ ]:
datasets['Entangled_video1_75000']['all_centers_dict_normed']['_1_MMStack_Pos0.ome.tif'][413L]['x'][140]

In [ ]:
data_cut = datasets['Entangled_video1_75000']['all_centers_dict_normed']['_1_MMStack_Pos0.ome.tif']

### 1. MSD: Histogram of displacements
You can change the lagtime in frames below. 

**Want to save all of the differences?**
You can draw out the values for whatever frame lagtime you want. If running multiple subdirectories together, save them under "All" condition or make a new folder with their combined data.

In [ ]:
lagtime_frames=1
plotAllHistOfDiffs(diffs_dict,lagtime_frames,binsize) 

In [ ]:
lagtime_frames=25
plotAllHistOfDiffs(diffs_dict,lagtime_frames,binsize) 

In [14]:
lagtime_value = [1, 2, 3, 5, 10, 15, 20, 25, 30, 40]
lagtime = ["1", "2", "3", "5", "10", "15", "20", "25", "30", "40"]
Axis='comb'
#Conformation="Ring"
#Condition = 'Actin'
#Date = 'All'
#mass_used="3000"
#group="All"

In [15]:

for i in (range(len(lagtime_value))):
    values=diffs_dict[Axis][lagtime_value[i]] #will give displacement values for each particle with lagtime_value frames diff
    w=open(f_root+"displacements_"+lagtime[i]+"frames".csv",'wb')
    writeFile=csv.writer(w)

    for n in values:
        writeFile.writerow([n])
    w.close()

In [22]:
len(diffs_dict['x'][1])

265641

 ### 2. Plotting MSDs: linear and log-log
 *Note* If you need to acccess further info within the MSD dictionary, averaged out MSD data is stored within msd_dict as a dictionary of dictionaries. The key-value pair is the axis *(x, y, comb)* paired with the dictionary of values for that axis. 
**I.E.** msd_dict['y'] gives you all the frame and pixel values of the averaged out MSD dictionary in the *y* frame

#### Linear

In [ ]:
plotAllMsds(msd_dict)

### Want to save the MSD values? Hint: yes you do. 
There are a couple different templates down there for where you can save .csv files to. You can also always just rewrite your own as well.

There are two saving options: one will save one day's worth of data to **that day** but the other will save **several days'** worth of data to **Condition_ALL** folder (i.e., when running all the data you have for one condition, you want to save it all together)

Recommended to save linear scale values and plot on log-log using Origin.

Axis can be 'x', 'y' or 'comb'

In [ ]:
axis='comb'
#day='050919' #if running only 1 day

In [ ]:
values=msd_dict[axis].values()

w=open(f_root+Conformation+"_"+mass_used+"\\"+Conformation+"_"+date+"_msd_vals_"+axis+".csv",'wb')
#w=open("\\\ANDERSONLAB\\AndersonLab\\"+User+"\\DATA\\DNA_in_XL_systems\\Lightsheet\\"+Conformation+"\\"+Condition+"\\Data\\"+Condition+"_ALL\\msd_vals_"+axis+".csv",'wb')
writeFile=csv.writer(w)

for n in values:
    writeFile.writerow([n])
w.close()


#### Log-Log

In [ ]:
plotAllMsds(msd_dict_ll, line_type='o')

#### Option to save Log-log values (not recommended)

In [ ]:
axis='comb'
day='042419' #if running only 1 day

In [ ]:
values=msd_dict_ll[axis].values()

w=open("\\\ANDERSONLAB\\AndersonLab\\Kathryn_R\\DATA\\DNA_in_XL_systems\\Lightsheet\\"+Conformation+"\\"+Condition+"\\Data\\"+Date+"_"+Condition+"_"+MassUsed+"\\LOG_msd_vals_"+axis+".csv",'wb')
#w=open("\\\ANDERSONLAB\\AndersonLab\\"+User+"\\DATA\\DNA_in_XL_systems\\Lightsheet\\"+Conformation+"\\"+Condition+"\\Data\\"+Condition+"_ALL\\LOG_msd_vals_"+axis+".csv",'wb')
writeFile=csv.writer(w)

for n in values:
    writeFile.writerow([n])
w.close()

### 3. Fitting MSDs: linear and log-log

You can tailor what frame range the fit is for, i.e. 0-24 is the first 25 frames
The fit coordinates that pop up are the coordinates for a line, *i.e.* **m,b** in **y=mx+b**

#### Linear:

In [ ]:
start_frame = 2
end_frame = 25
linear_fits = linearFitAllAxes(msd_dict, start_frame=start_frame, end_frame=end_frame, plot=True)
print linear_fits

#### Log-Log:

In [ ]:
start_frame = 2
end_frame = 25
fits = linearFitAllAxes(msd_dict_ll, start_frame=start_frame, end_frame=end_frame, line_type='-o')
print(fits)

## Conformational Methods

### Basic conformational functions needed

In [ ]:
def calculateConformationsForDatasets(datasets, f_root, subdir, plot=True):
    all_r_max_min = {}
    r_max=[] #1.22.19
    r_min=[] #1.22.19
    for subdir_name, dataset in datasets.items():
        print "Calculating conformations for: %s" % subdir_name
        
        # get list of videos to pull
        centered_frames_loc = os.path.join(f_root, subdir_name, 'centered_frames')
        print subdir_name
        print centered_frames_loc
        videos =  [video for video in os.listdir(centered_frames_loc) if video.endswith('.p')]
        print videos
        for video in videos:
            data = pickle.load(open(os.path.join(centered_frames_loc, video), "rb" ))
            all_r_max_min[video] = {}
            for particle, img_stack in data.items():
                r_max_min = calculateRMaxMin(img_stack, f_root, subdir_name, video, particle, plot=plot)
                all_r_max_min[video][particle] = r_max_min                
                

        r_max_min_loc = os.path.join(f_root, subdir_name, 'r_max_min.p')
        pickle.dump(all_r_max_min, open(r_max_min_loc, 'wb'))
        print "\tFinished dataset!"
        
def loadMultipleConformationalDatasets(f_root, subdir_names):
    all_r_max_min = {}
    for subdir_name in subdir_names:
        r_max_min_loc = os.path.join(f_root, subdir_name, 'r_max_min.p')
        print r_max_min_loc
        all_r_max_min[subdir_name] = pickle.load(open(r_max_min_loc, "rb" ))
    return all_r_max_min

### Methods for calculating R_max, R_min

#### Load files needed

In [ ]:
def loadIndividualDataset(f_root, subdir_name):
    ''' Loads an individual dataset
    
    Input:
        f_root = root directory for data
        subdir_name = name of the subdirectory containing data
        frames (optional) = option to grab frames data
        
    Output:
        centers = dictionary containing data for the individual dataset
    '''
    sub_dir = os.path.join(f_root, subdir_name)
    
    centers_loc = os.path.join(sub_dir, 'centers.p')
    centers = pickle.load(open(centers_loc, "rb" ))

    return centers

def loadMultipleDatasets(f_root, subdir_names):  
    ''' Loads multiple datasets into dict
    
    Input:
        f_root = root directory for data
        subdir_names = list of subdirectories to pull data for
        frames (optional) = option to grab frames data
    '''
    return {subdir_name: loadIndividualDataset(f_root, subdir_name) \
            for subdir_name in subdir_names}


#### Calculate Rmax, Rmin

In [ ]:
def calculateImageMean(image):
    vals = []
    for row in image:
        vals.extend(row)

    return mean(vals)

def calculateImageThreshold(bkgd_mean):
    ''' Calculate threshold for cutting off background. Very simple to 
    start, just greater than image mean 
    
    '''
    return bkgd_mean + 30
    #return (255+bkgd_mean)/2.0

def getLargestContour(contours):
    # limit to largest contour (particle)
    areas = []
    for contour in contours:
        ar = cv2.contourArea(contour)
        areas.append(ar)
    
    max_area = max(areas)
    max_area_index = areas.index(max_area)
    return contours[max_area_index]

def euclideanDistance(point_1, point_2):
    return ((point_1[0] - point_2[0])**2. + (point_1[1] - point_2[1])**2.)**0.5

def calculateRMaxMin(img_stack, f_root, subdir_name, video, particle, plot=True):
    r_max_min = {}
    
    for frame, img in enumerate(img_stack):
        bkgd_mean = calculateImageMean(img)
        cutoff_val = calculateImageThreshold(bkgd_mean)
        ret, thresh1 = cv2.threshold(img,cutoff_val,255,cv2.THRESH_BINARY)

        # find object outlines
        _, contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) 
        #print contours

        # limit to biggest contour
        primary_contour = getLargestContour(contours)

        # get rectangle to approximate the contour
        rect = cv2.minAreaRect(primary_contour)
        box = cv2.boxPoints(rect)

        # get midpoints from box vertices
        anchor = box[0]
        dists = []
        for i, vertex in enumerate(box[1:]):
            dists.append((euclideanDistance(anchor, vertex), i+1))

        dists = sorted(dists)
        len_min_axis = dists[0][0]
        len_maj_axis = dists[1][0]

        minor_vertex = box[dists[0][1]]
        major_vertex = box[dists[1][1]]
        anchor_2 = box[dists[2][1]]

        delta_minor = ((anchor[0] - minor_vertex[0])/2., (anchor[1] - minor_vertex[1])/2.)
        delta_major = ((anchor[0] - major_vertex[0])/2., (anchor[1] - major_vertex[1])/2.)

        minor_mid_1 = anchor[0]-delta_minor[0], anchor[1]-delta_minor[1]
        minor_mid_2 = anchor_2[0]+delta_minor[0], anchor_2[1]+delta_minor[1]

        major_mid_1 = anchor[0]-delta_major[0], anchor[1]-delta_major[1]
        major_mid_2 = anchor_2[0]+delta_major[0], anchor_2[1]+delta_major[1]

        # add each frame to the dict
        r_max_min[frame] = {'major': (major_mid_1, major_mid_2),
                           'minor': (minor_mid_1, minor_mid_2),
                           'r_max': len_maj_axis,
                           'r_min': len_min_axis,}

        # draw each image with Rmax/Rmin superimposed
        if plot:
            plt.imshow(img);
            plt.grid(False);
            plt.plot((minor_mid_1[0], minor_mid_2[0]), (minor_mid_1[1], minor_mid_2[1]), 'k-', color='white');
            plt.plot((major_mid_1[0], major_mid_2[0]), (major_mid_1[1], major_mid_2[1]), 'k-', color='white');
            loc_to_write = os.path.join(f_root, subdir_name, 'r_max_min/%s_%d_%d.png' % (video, particle, frame))
            plt.savefig(loc_to_write);
            plt.clf();
        
    return r_max_min


### Methods for calculating Fluctuation Length L(t) = <|Rmax(t+tau) - Rmax(t)|>

In [ ]:
''' Methods for plotting L(t) = <abs(Rmax(t+tau) - Rmax(t))>
'''

def getRmaxDiffs(all_r_max_min):
    ''' Takes dictionary (output of `loadMultipleConformationalDatasets`) and iterates through each particle.
    For each particle, sweeps through and finds all diffs between frames for r_max (for each possible window
    size) and adds to master dict (one for each axis)
    
    Input:
        all_r_max_min = dictionary of conformational datasets, output of `loadMultipleConformationalDatasets`
        
    Output:
    
        dictionary of r_max diffs for each window size
    '''
    diffs_r_max = defaultdict(list)
    r_max_save = {}
    # iterate through datasets
    for dataset, data in all_r_max_min.iteritems():
        # iterate through videos
        for video, particles in data.items():
            # iterate through particles
            for particle, conform_data in particles.items():
                r_max_vals = [conform_data[frame]['r_max'] for frame in sorted(conform_data.keys())]
                  

                # iterate through possible sliding window sizes
                for window in range(0, len(r_max_vals)):
                    for ind_start in range(len(r_max_vals)-window):
                        diffs_r_max[window].append(abs(r_max_vals[window+ind_start] - r_max_vals[ind_start])) 

    return diffs_r_max 


def calculateFluctuationValues(diffs_r_max):
    ''' Calculates L(t) = <abs(Rmax(t+tau) - Rmax(t))> for tau = 0:numberOfFrames
    
    Input:
        diffs_r_max = dictionary of r_max diffs, output of `getRmaxDiffs`
        
    Output:
        dictionary, where key=tau, value=<abs(Rmax(t+tau) - Rmax(t))>
    '''
    
    return {window: mean(diffs) for window, diffs in diffs_r_max.iteritems()}

def plotFluctuationValues(fluct_values, plateau_range=None):
    ''' Plots L(t) vs t (in frames)
    
    Input:
        fluct_values = output from `calculateFluctuationValues`
        plateau_range (optional) = if provided, calculates plateau by taking average
                                    over frame range provided [start, end]
    Output:
        None
    '''
    plt.plot(fluct_values.keys(), fluct_values.values(), '-o')
    plt.xlabel('frame', fontsize=15);
    plt.ylabel('<abs(Rmax(t+tau) - Rmax(t))> (pixels)', fontsize=15);
    
    #fluctvals=fluct_values.values()
    #return fluctvals
    
    if plateau_range:
        plateau = mean(fluct_values.values()[plateau_range[0]:plateau_range[1]])
        print 'Plateau = %.6f pixels' % plateau
        plt.plot([fluct_values.keys()[0], fluct_values.keys()[-1]], [plateau, plateau], 'k-');


### 1. Calculate Rmax and Rmin values
Calculates R_max and R_min for each frame using the centered videos created in Particle Tracking step. Results in "all_r_max_min", a dictionary containing the length of R_max and R_min for each frame along with the points for drawing both axes. 

Can also plot the lines of major/minor axis on each frame but takes significantly longer. Leave "plot=False" for **no plotting** or leave "plot=True" to **dump images** with axes overlayed for every frame to disk. Useful for confirming results (and troubleshooting weird results). In the past, I've set "plot=True" but then interrupt the kernel after maybe 45s of plotting -- you'll still get a good collection of images to look at and trouble check. Stored in 'r_max_min' folder in parent data folder for the day/condition being analyzed

In [ ]:
#Conformation = "Ring"
#Condition = "Actin"
#Date = "041519"

#f_root = "\\\ANDERSONLAB\\AndersonLab\\Kathryn_R\\DATA\\DNA_in_XL_systems\\
#Lightsheet\\"+Conformation+"\\"+Condition+"\\Data\\"
#subdir_names=[Conformation+Condition+"_"+group+"of10_1000"]
datasets = loadMultipleDatasets(f_root, subdir_names)
calculateConformationsForDatasets(datasets, f_root, subdir_names, plot=False)


# load results dict
all_r_max_min = loadMultipleConformationalDatasets(f_root, subdir_names)

#### Want to save your R_max, R_min values? Hint: you probably do

In [ ]:
#Date="050919"
#mass_used="2000"

In [ ]:

#f_root = "\\\ANDERSONLAB\\AndersonLab\\Kathryn_R\\DATA\\DNA_in_XL_systems\\Lightsheet\\"+Conformation+"\\"+Condition+"\\Data\\"

rmax=[]
rmin=[]


for subdir_name in subdir_names:
    centered_frames_loc = os.path.join(f_root, subdir_name, 'centered_frames')
    videos =  [video for video in os.listdir(centered_frames_loc) if video.endswith('.p')]
    num1=subdir_name
    for video in videos:
        num2 = video
        r=len(all_r_max_min[num1][num2].keys())
        particle=all_r_max_min[num1][num2].keys()
        #print r
        #print particle
        for listing in range(0,r):
            num3=particle[listing]
            tracked_length=(all_r_max_min[num1][num2][num3].keys())
            length2=len(tracked_length)
            for num4 in range(0,length2):
                r_max=all_r_max_min[num1][num2][num3][num4].values()[1]
                rmax.append(r_max)
                r_min=all_r_max_min[num1][num2][num3][num4].values()[2]
                rmin.append(r_min)



w=open(f_root+Conformation+Condition+"_"+group+"_"+Date+"\\"+Conformation+Condition+"_"+group+"_"+Date+"_r_max_values.csv",'wb')
writeFile=csv.writer(w)
for n in rmax:
    writeFile.writerow([n])
w.close()

t=open(f_root+Conformation+Condition+"_"+group+"_"+Date+"\\"+Conformation+Condition+"_"+group+"_"+Date+"_r_min_values.csv","wb")
writeFile=csv.writer(t)
for m in rmin: 
    writeFile.writerow([m])
t.close()


### 2. Calculating Fluctuation Values L(t)

Calculates L(t) = <|Rmax(t+tau) - Rmax(t)|> from all_r_max_min dictionary, calculated at start of conformational analysis


In [ ]:
plateau_range = [25,35]

diffs_r_max = getRmaxDiffs(all_r_max_min)
fluct_values = calculateFluctuationValues(diffs_r_max)
plotFluctuationValues(fluct_values, plateau_range=plateau_range)


In [ ]:
fluctvals=fluct_values.values()
frames=fluct_values.keys()


w=open(f_root+Conformation+Condition+"_"+group+"_"+Date+"\\"+Conformation+Condition+"_"+group+"_"+Date+"_Fluct_Vals.csv",'wb')
writeFile=csv.writer(w)

for n in fluctvals:
    writeFile.writerow([n])
    
w.close()

#'''
#f=open(f_root+"Data\\\\frames.csv",'wb')
#writeFile=csv.writer(f)

#for n in frames:
 #   writeFile.writerow([n])
    
#w.close()
#'''

### 3. Dot Product Measurements
**USES RMAX, RMIN VALUES YOU JUST RAN**

#### Functions used

In [ ]:
def calculateDotProduct(delta_1, delta_2):
    ''' Calculates dot product between two vectors with base (0, 0)
    
    Input:
        delta_1 = (x_1, y_1)
        delta_2 = (x_2, y_2)
        
    Output:
        delta_1 __dot__ delta_2
    '''
    total = 0
    for i, j in zip(delta_1, delta_2):
        total += i*j
        
    return total

def calculateAngleFromDeltas(delta):
    ''' calculates angle from vector with base (0, 0)
    
    Input:
        delta = (x, y)
        
    Output:
        arctan(y/x)
    '''
    return math.atan(delt[1]/float(delt[0]))

def calculateDeltas(data, major_or_minor):
    ''' normalize 2 points into single point with base (0, 0)
    and length 1
    
    Input:
        delta = ((x_1, y_1), (x_2, y_2))
        
    Output:
        (x_new, y_new)
    '''
    dmy = data[major_or_minor]
    delt_x = dmy[0][0] - dmy[1][0]
    delt_y = dmy[0][1] - dmy[1][1]
    norm_factor = (delt_x**2 + delt_y**2)**0.5
    
    return (delt_x/norm_factor, delt_y/norm_factor)
    

In [ ]:
def getDotProducts(all_r_max_min, major_or_minor):
    ''' Calculates dot products for all potential time windows. Similar
    to `getRmaxDiffs` method. 
    
    Input:
        all_r_max_min = dictionary of conformational datasets, output of `loadMultipleConformationalDatasets`
        major_or_minor = 'major' or 'minor' axis
        
    Output:
        dictionary of dot products for each window size        
    '''
    if major_or_minor not in ('major', 'minor'):
        raise Exception("Only 'major' or 'minor' allowed for `major_or_minor` arg. Received %s")
    
    dot_prods = defaultdict(list)
    
    # iterate through datasets
    for dataset, data in all_r_max_min.iteritems():
        # iterate through videos
        for video, particles in data.items():
            # iterate through particles
            for particle, conform_data in particles.items():
                axes_data = [calculateDeltas(data, major_or_minor) for data in conform_data.values()]
                
                # iterate through possible sliding window sizes
                for window in range(0, len(axes_data)):
                    for ind_start in range(len(axes_data)-window):
                        dot_prod = calculateDotProduct( \
                                axes_data[window+ind_start], axes_data[ind_start])
                        
                        if np.isnan(dot_prod):
                            continue
                        
                        dot_prods[window].append(dot_prod)
                             
    return dot_prods

def meanDotProds(dot_prods):
    ''' Calculates mean dot product for each frame
    
    Input:
        dot_prods = output from `getDotProducts`
        
    Output:
        dict of mean dot products {frame: mean_dot_product}
        
    '''
    return {frame: mean(prods) for frame, prods in dot_prods.iteritems()}

def plotDotProds(dot_prods, major_or_minor):
    ''' Plots dot products data
    
    Input:
        dot_prods = output from `meanDotProds`
        major_or_minor = 'major' or 'minor' axis
        
    Output:
        None
    '''
    
    plt.figure();
    plt.plot(dot_prods.keys(), dot_prods.values(), '-o');
    plt.xlabel('frame', fontsize=20);
    plt.title('%s axis' % major_or_minor, fontsize=20);
    plt.ylabel('<%s(t+tau) __dot__ %s(t)>' % (major_or_minor, major_or_minor), fontsize=20)
    

#### Calculating, Saving Dot products

In [ ]:
major_or_minor = 'major'
dot_prods = getDotProducts(all_r_max_min, major_or_minor)
mean_dot_prods = meanDotProds(dot_prods)
plotDotProds(mean_dot_prods, major_or_minor)

mean_dotprods=mean_dot_prods.values()
time=mean_dot_prods.keys()

#Date= "050919"
w=open(f_root+Conformation+Condition+"_"+group+"_"+Date+"\\"+Conformation+Condition+"_"+group+"_"+Date+"_mean_dot_prod_major.csv",'wb')
writeFile=csv.writer(w)

for n in mean_dotprods:
    writeFile.writerow([n])
    
w.close()



In [ ]:
major_or_minor = 'minor'
dot_prods = getDotProducts(all_r_max_min, major_or_minor)
mean_dot_prods = meanDotProds(dot_prods)
plotDotProds(mean_dot_prods, major_or_minor)

mean_dotprods=mean_dot_prods.values()
time=mean_dot_prods.keys()

#Date= "042419"
w=open(f_root+Conformation+Condition+"_"+group+"_"+Date+"\\"+Conformation+Condition+"_"+group+"_"+Date+"_mean_dot_prod_minor.csv",'wb')
writeFile=csv.writer(w)

for n in mean_dotprods:
    writeFile.writerow([n])
    
w.close()




## Ergodicity

### Ergodicity Functions

In [ ]:
def calculateMsdByParticle(datasets):
    ''' Calculates MSD for each individual particle, rather than combining across
    all particles and then calculating MSD (like in `calculateMSDs` method)
    
    Input:
        datasets = output from `loadMultipleDatasets`
    
    Output:
        d_2_dict = dict of individual MSDs
    '''
    d_2_dict = {'x': [],
                'y': [],
                'comb': [],}

    # iterate through datasets
    for dataset, data in datasets.iteritems():
        # iterate through videos
        for video, particles in data['all_centers_dict_normed'].items():
            # iterate through particles
            for particle, center_data in particles.items():
                x_centers = center_data['x']
                y_centers = center_data['y']

                x_msd = {}
                y_msd = {}
                c_msd = {}
                # iterate through possible sliding window sizes
                for window in range(1, len(x_centers)):
                    x_msd[window] = []
                    y_msd[window] = []
                    c_msd[window] = []
                    for ind_start in range(len(x_centers)-window):
                        x_msd[window].append(x_centers[window+ind_start] - x_centers[ind_start]) 
                        y_msd[window].append(y_centers[window+ind_start] - y_centers[ind_start])
                        c_msd[window].append(x_msd[window][-1])
                        c_msd[window].append(y_msd[window][-1])

                dmy_dict = {'x': x_msd,
                           'y': y_msd,
                           'comb': c_msd}

                # calculate MSD for individual particle
                dmy_dict_msd = calculateMSDs(dmy_dict)
                d_2_dict['x'].append(dmy_dict_msd['x'])
                d_2_dict['y'].append(dmy_dict_msd['y'])
                d_2_dict['comb'].append(dmy_dict_msd['comb'])

    return d_2_dict

def squareIndividualMsds(d_2_dict):
    ''' Square each of the individual MSDs
    
    Input:
        d_2_dict = output from `calculateMsdByParticle`
    
    Output:
        d_2_dict_2 = dict of individual MSDs squared
    '''
    d_2_dict_2 = {'x': [],
                  'y': [],
                  'comb': [],}

    for axis, particles in d_2_dict.items():
        for particle in particles:
            new_dict = {}
            for frame, px_2 in particle.items():
                new_dict[frame] = px_2**2

            d_2_dict_2[axis].append(new_dict)
            
    return d_2_dict_2

def averageAcrossMsds(d_2_dict_2):
    ''' Average across each of the individual MSDs, tracking
    how many particles exist for each frame number (ie, every one of
    the N particles will have a frame 1 and need to be normalized 
    by N, but not every particle will extend out to frame f_max. If
    only 1 particle is f_max frames long, then the average is simply the
    single value at f_max/1, NOT divided by N).
    
    Input:
        d_2_dict_2 = output from `squareIndividualMsds`
    
    Output:
        d_2_dict_2_average = dict of averaged values across d_2_dict_2
    '''
    def _emptyList():
        return [0, 0]

    avg_across_particles = {'x': defaultdict(_emptyList),
                            'y': defaultdict(_emptyList),
                            'comb': defaultdict(_emptyList),}

    for axis, particles in d_2_dict_2.items():
        for particle in particles:
            for frame, px_4 in particle.items():
                avg_across_particles[axis][frame][0] += 1
                avg_across_particles[axis][frame][1] += px_4

    d_2_dict_2_average = {}
    for axis, data in avg_across_particles.iteritems():
        d_2_dict_2_average[axis] = {}
        for frame, frame_data in data.items():
            d_2_dict_2_average[axis][frame] = frame_data[1]/float(frame_data[0])

    return d_2_dict_2_average


def squareStandardMsds(msd_dict):
    ''' Squares standard MSD values to get second term required
    to calculate EB
    
    Input:
        msd_dict = output from `calculateMSDs`
    
    Output:
        msd_dict_2 = msd_dict with each value squared
    '''
    msd_dict_2 = {'x': [],
                  'y': [],
                  'comb': [],}

    for axis, frame_data in msd_dict.items():
        new_dict = {}
        for frame, px_2 in frame_data.items():
            new_dict[frame] = px_2**2

        msd_dict_2[axis] = new_dict
        
    return msd_dict_2

def calculateEB(d_2_dict_2_average, msd_dict_2):
    ''' Calculates EB from the values required for the numerator +
    denominator by frame as defined in equation 15 from:
    
    Nonergodicity, fluctuations, and criticality in heterogeneous diffusion processes
    PHYSICAL REVIEW E 90, 012134 (2014)
    
    Input:
        d_2_dict_2_average = output from `averageAcrossMsds`
        msd_dict_2 = output from `squareStandardMsds`
    
    Output:
        eb = eb by frame (for each axis)
    '''
    eb = {}
    for axis in ('x', 'y', 'comb'):
        eb[axis] = {}
        for frame, val in d_2_dict_2_average[axis].iteritems():
            if frame == 0:
                continue 

            term_1 = val
            term_2 = msd_dict_2[axis][frame]
            eb[axis][frame] = (term_1 - term_2)/float(term_2)

    return eb

def calculateEBFromDatasets(datasets):
    ''' Runs through entire process to calculate and return EB(frame)
    from just datasets.
    
    Input:
        datasets = output from `loadMultipleDatasets`
        
    Output:
        eb = eb by frame (for each axis)
    '''
    d_2_dict = calculateMsdByParticle(datasets)
    d_2_dict_2 = squareIndividualMsds(d_2_dict)
    d_2_dict_2_average = averageAcrossMsds(d_2_dict_2)

    diffs_dict = getDiffs(datasets)
    msd_dict = calculateMSDs(diffs_dict)
    msd_dict_2 = squareStandardMsds(msd_dict)
    
    return calculateEB(d_2_dict_2_average, msd_dict_2)

def calculateGParameterFromDatasets(datasets, d=1):
    ''' Runs through entire process to calculate and return G(frame)
    from just datasets as defined by equation 19 from:
    
    Nonergodicity, fluctuations, and criticality in heterogeneous diffusion processes
    PHYSICAL REVIEW E 90, 012134 (2014)
    
    Input:
        datasets = output from `loadMultipleDatasets`
        
    Output:
        G = G by frame (for each axis)
    '''
    diffs_dict = getDiffs(datasets)
    msd_dict = calculateMSDs(diffs_dict)
    denom_dict = squareStandardMsds(msd_dict)

    squared_diffs_dict = {}
    for axis, axis_dict in diffs_dict.iteritems():
        squared_diffs_dict[axis] = {}
        for window, diffs in axis_dict.iteritems():
            squared_diffs_dict[axis][window] = [diff**2 for diff in diffs]

    numer_dict = calculateMSDs(squared_diffs_dict)           

    G = {}
    for axis, numer_data in numer_dict.iteritems():
        denom_data = denom_dict[axis]
        G[axis] = {frame: d/(d+2.) * numer_data[frame]/float(denom_data[frame]) - 1 \
               for frame in numer_data.keys() if frame > 0}

    return G

def plotSingleAxis(single_data_dict, axis, y_label):
    ''' Takes a dictionary of data, designed for both
    eb, G and plots a single axis.
    
    Input:
        single_data_dict =  dict of data to be plotted
        axis = string to clarify which axis is being plotted
        y_label = string to label the y-axis
        
    Output:
        None
    '''
    plt.figure();
    plt.plot(single_data_dict.keys(), single_data_dict.values(), '-o');
    plt.xlabel('frame', fontsize=20)
    plt.ylabel(y_label, fontsize=20)
    plt.title('Axis = %s' % axis, fontsize=20)
    
def plotAllAxes(data_dict, y_label):
    ''' Takes a dictionary of data, designed for both
    eb, G and plots all axes.
    
    Input:
        data_dict =  dict of data to be plotted (designed for G or eb)
        y_label = string to label the y-axis
        
    Output:
        None
    '''
    for axis, single_data_dict in data_dict.iteritems():
        plotSingleAxis(single_data_dict, axis, y_label)


In [ ]:
datasets = loadMultipleDatasets(f_root, subdir_names)
eb = calculateEBFromDatasets(datasets)
#plotAllAxes(eb, 'EB(frame)')
#plt.show()

G = calculateGParameterFromDatasets(datasets)
#plotAllAxes(G, 'G(frame)')
#plt.show()

In [ ]:
#G = calculateGParameterFromDatasets(datasets)
plotAllAxes(G, 'G(frame)')

In [ ]:
#Conformation = "Ring"
#Condition = "CoXL"
axis = "comb"
EB=[]
#Date="052119"
for v in range(0,390):
    EBValues_List=eb[axis].values()
    Add_EBVal=EBValues_List[v]
    EB.append(Add_EBVal)
    
w=open(f_root+Conformation+"_"+mass_used+"\\"+Conformation+date+"_EBvalues.csv","wb")
writeFile=csv.writer(w)

for n in EB:
    writeFile.writerow([n])
    
w.close()


In [ ]:
axis = "comb"
Gvals=[]

for v in range(0,390):
    Gvalues_List=G[axis].values()
    Add_Gval=Gvalues_List[v]
    Gvals.append(Add_Gval)
    
w=open(f_root+Conformation+"_"+mass_used+"\\"+Conformation+date+"_Gvalues.csv","wb")
writeFile=csv.writer(w)

for n in Gvals:
    writeFile.writerow([n])
    
w.close()


In [ ]:
test=calculateMsdByParticle(datasets)

In [ ]:
w=open(f_root+Conformation+"_"+mass_used+"\\"+Conformation+date+"_SingleTracks.csv", "wb")
fieldnames = range(1999)
writeFile=csv.DictWriter(w,fieldnames,extrasaction='ignore')
for i in range(len(test['comb'])):
    writeFile.writerow(test['comb'][i])
w.close()